In [1]:
import pandas as pd

# Load the Excel file into a DataFrame
df_bridges_all = pd.read_excel("bridges_data_p2.xlsx")

In [2]:
import geopandas as gpd
# Replace the file path with your actual file path
# region_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # doma
region_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\regions_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
region_gdf = gpd.read_file(region_path)


# districts_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # doma
districts_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\districts_epsg_4326.geojson'  # robota

# Load the GeoJSON file into a GeoDataFrame
districts_gdf = gpd.read_file(districts_path)


# Replace the file path with your actual file path
# file_path = r'C:\Users\Doktor\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # doma
file_path = r'C:\Users\relia\Documents\GitHub\Bridges\GeoJSONs\cities_regions_districts_epsg_4326.geojson'  # doma


# Load the GeoJSON file into a GeoDataFrame
settle_gdf = gpd.read_file(file_path)

# Define the types to exclude
types_to_exclude = ['country', 'airport', 'Psie skaly',
                    'county', 'Drevenný kostol', 'Vodopád']

# Create a boolean mask to select rows where the 'type' column is not in types_to_exclude
# and the 'population' column is not null
mask = (~settle_gdf['type'].isin(types_to_exclude)) & (
    settle_gdf['population'] > 0)


# Apply the mask to filter the GeoDataFrame in place
settle_gdf = settle_gdf[mask]

In [12]:
import numpy as np

# Define a function to select a random bridge from each group


def random_bridge(group):
    # Select one random sample from each group
    return group.sample(1, random_state=420)


# Group by "Kraj" and apply the random_bridge function
random_bridges = df_bridges_all.groupby(
    "Kraj", group_keys=False).apply(random_bridge)

# Reset index to make it a DataFrame
random_bridges = random_bridges.reset_index(drop=True)

# Display the randomly selected bridges
random_bridges

,ID_mosta,Trieda_PK,Číslo_PK,Názov_mosta,správcovské_číslo,Zemepisná_dĺžka,Zemepisná_šírka,Výška,Druh_konštrukcie,Material,...,n_2016,n_2017,n_2018,n_2019,n_2020,n_2021,n_2022,n_2023,Trieda_PK_n,Okres_n
0,M35,cesta III. triedy,2749,2749-1 (050126-001) Most cez potok Pápča v obc...,1,20.064702,48.468025,227.524500,dosková,monolitický železobetón,...,2,2,2,2,2,4,4,4,4,609
1,M4498,diaľnica,D1,Most na vetve 3 križovatky Gagarinova,023A4,17.181763,48.151013,186.041249,Nezistený,Nezistený,...,0,0,0,0,0,0,2,2,1,102
2,M5854,cesta III. triedy,3799,Mosty cez potok Slaný pred obcou Baškovce,1,22.201606,48.774259,153.332750,dosková,prefabrikovaný železobetón,...,3,3,3,3,3,3,3,6,4,809
3,M5990,cesta III. triedy,1681,Most v Šuriankach cez potok,2,18.023612,48.418212,154.418999,dosková,monolitický železobetón,...,5,5,5,5,5,6,6,6,4,403
4,M2646,cesta III. triedy,3099,Most cez Holumnický potok v obci Holumnica,3,20.520148,49.238581,586.979250,trámová,oceľ - plnostenná,...,4,4,4,4,4,5,6,6,4,703
5,M5751,cesta II. triedy,499,Most cez rieku Myjava v meste Myjava,16,17.562702,48.747390,315.321249,dosková,prefabrikovaný predpätý betón,...,4,3,3,3,3,4,4,4,3,303
6,M729,cesta III. triedy,1395,1395-001 most cez Klátovské rameno v obci Trho...,1,17.750381,47.992229,113.950500,dosková,prefabrikovaný predpätý betón,...,4,4,4,4,4,4,4,5,4,201
7,M3606,cesta I. triedy,65,Most 65_088 cez Blatnícky potok pri obci Príbovce,88,18.890739,48.993380,431.534499,dosková,prefabrikovaný predpätý betón,...,1,1,1,1,1,1,1,1,2,506


In [15]:
random_bridges.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'Trieda_PK_n', 'Okres_n'],
      d

In [14]:
settle_gdf.iloc[69]

osm_id                             26035766
name                              Martinček
region                             Žilinský
district                         Ružomberok
type                                village
population                              394
geometry      POINT (19.3323344 49.0901132)
Name: 79, dtype: object

In [16]:
import geopandas as gpd
from shapely.geometry import Point

In [17]:
def find_nearby_settlements(bridge_row, settlements_gdf, radius_km=[1, 5, 15]):
    # Create a point geometry for the bridge
    bridge_point = Point(
        bridge_row['Zemepisná_dĺžka'], bridge_row['Zemepisná_šírka'])

    # Initialize lists to hold the names of settlements within each distance range
    settlements_within_1km = []
    settlements_within_1_to_5km = []
    settlements_within_5_to_15km = []

    # Calculate the distance from the bridge to each settlement in kilometers
    for _, settle_row in settlements_gdf.iterrows():
        # Adjust the multiplication factor based on your coordinate system
        distance_km = bridge_point.distance(settle_row['geometry']) * 100

        # Categorize the settlement based on the distance
        if distance_km <= radius_km[0]:
            settlements_within_1km.append(settle_row['name'])
        elif radius_km[0] < distance_km <= radius_km[1]:
            settlements_within_1_to_5km.append(settle_row['name'])
        elif radius_km[1] < distance_km <= radius_km[2]:
            settlements_within_5_to_15km.append(settle_row['name'])

    # Return a Series with lists of settlements for each distance range
    return pd.Series([settlements_within_1km, settlements_within_1_to_5km, settlements_within_5_to_15km],
                     index=['within_1km', 'within_1_to_5km', 'within_5_to_15km'])

In [18]:
# Apply the function to each bridge
nearby_settlements = random_bridges.apply(
    lambda row: find_nearby_settlements(row, settle_gdf), axis=1)

# Concatenate the results with the original random_bridges dataframe
random_bridges_with_settlements = pd.concat(
    [random_bridges, nearby_settlements], axis=1)

In [45]:
random_bridges_with_settlements.columns

Index(['ID_mosta', 'Trieda_PK', 'Číslo_PK', 'Názov_mosta', 'správcovské_číslo',
       'Zemepisná_dĺžka', 'Zemepisná_šírka', 'Výška', 'Druh_konštrukcie',
       'Material', 'Normálna_zaťažiteľnosť', 'Výhradná_zaťažiteľnosť_t',
       'Výnimočná_zaťažiteľnosť_t', 'Spôsob_stanovenia', 'Predmet_premostenia',
       'Rok_postavenia', 'Zaťažovacia_trieda_STN', 'Počet_otvorov',
       'Dĺžka_premostenia', 'Dĺžka_nosnej_konštrukcie_m', 'Volná_šírka_mosta',
       'Šírka_medzi_obrubami_m', 'Plocha_mosnej_konštrukcie_m2',
       'Plocha_mosta_m2', 'Vlastník_úseku', 'Správca_úseku', 'Okres',
       'Smer_úseku', 'Typ_úseku', 'Počiatočný_uzol', 'Koncový_uzol', 'úsek_ID',
       'úsekové_staničenie_začiatku_DC_m',
       'Kilometrovníkové_staničenie_začiatku_DC_km',
       'Kumulatívne_staničenie_začiatku_DC_km', 'Poradie', 'Kraj', 'n_2012',
       'n_2013', 'n_2014', 'n_2015', 'n_2016', 'n_2017', 'n_2018', 'n_2019',
       'n_2020', 'n_2021', 'n_2022', 'n_2023', 'Trieda_PK_n', 'Okres_n',
       '

In [55]:
cislo = 0
random_bridges_with_settlements.iloc[cislo]["Kraj"], random_bridges_with_settlements.iloc[cislo]["within_1km"], random_bridges_with_settlements.iloc[
    cislo]["within_1_to_5km"], random_bridges_with_settlements.iloc[cislo]["within_5_to_15km"]

('Banská Bystrica',
 [],
 ['Hrušovo',
  'Padarovce',
  'Drienčany',
  'Teplý Vrch',
  'Budikovany',
  'Dražice',
  'Lukovištia'],
 ['Lipovec',
  'Rybník',
  'Vyšné Valice',
  'Rimavské Zalužany',
  'Kraskovo',
  'Sása',
  'Zacharovce',
  'Skerešovo',
  'Hostišovce',
  'Chvalová',
  'Uzovská Panica',
  'Ratkovská Suchá',
  'Rimavská Baňa',
  'Rimavská Sobota',
  'Pavlovce',
  'Čerenčany',
  'Bátka',
  'Rybník',
  'Hrachovo',
  'Rimavské Brezovo',
  'Ratková',
  'Rimavské Janovce',
  'Ratkovská Lehota',
  'Veľký Blh',
  'Nižný Skálnik',
  'Horné Zahorany',
  'Višňové',
  'Kyjatice',
  'Slizké',
  'Tomášovce',
  'Kociha',
  'Sútor',
  'Španie Pole',
  'Belín',
  'Veľké Teriakovce',
  'Vyšný Skálnik',
  'Kružno',
  'Babinec',
  'Potok',
  'Rovné',
  'Rakytník'])